In [4]:
import numpy
import nibabel as nib
import os
import matplotlib.pyplot as plt
# Importing our own functions
from functions.volume import volume
from functions.label import SegmentationLabel

# Example running the code
In this notebook will guide on how to use a segmentation file and obtain MR property volumes from it

In [5]:
# Inside the example folder we have already segmented files from CT
# dataset by Gatidis et al. https://doi.org/10.1038/s41597-022-01718-3
# Attribution to the data set link : https://creativecommons.org/licenses/by/4.0/
ct_wb_seg = nib.load("example/final_total_seg.nii.gz") # wb stands for whole body
# We can acquire the numpy data from the nifti
labels = ct_wb_seg.get_fdata()
# And we can quickly see the shape of our data
labels.shape

(400, 400, 326)

# MR Property Phantom Creation

In [7]:
# Here we call the Volume function
# This volume function works for whole body - Total Seg CT
new_vol = volume(ct_wb_seg)

In [9]:
# Given that we know that all the labels from Total Seg should be the same
# We can set all the labels automatically with a specific method
new_vol.create_segmentation_labels("TotalSegmentator","v2")

Label ID 256 not found.
Label ID 256 not found.
Label ID 289 not found.
Label ID 289 not found.


In [ ]:
new_vol.group_seg_labels("TotalSegmentator","v2")

This Labels not found are pre-defined labels that work with final_sc_seg.nii.gz </br>
This file will be used later in the example of creating new labels :)

In [6]:
%%time
# Now we have all the labels from the whole body stablished
# By default T2star and Susceptibility values are complete 
# You can check the values used in the look-up table on the README file
sus_dist = new_vol.create_sus_dist()
# The output will be a Numpy array 

CPU times: total: 3.55 s
Wall time: 16.9 s


In [7]:
# A method is created to save the susceptibility from Numpy array to Nifti file
new_vol.save_sus_dist_nii()
# By default it will save it in the "output" folder

In a similar way we can create proton density and T2star volumes, acquire their numpy type data and then save them as nifti files

In [8]:
%%time
pd_vol = new_vol.create_pd_vol()
t2_star_vol = new_vol.create_t2_star_vol()

CPU times: total: 5.05 s
Wall time: 31.9 s


In [9]:
new_vol.save_pd_dist()
new_vol.save_t2star_dist()

Now you will be able to open any of this 3 volumes on a viewer of choice. </br>
As mentioned in the README file, if you use ITK snap you can open the segmentation labels.